##### 

In [1]:
%pip install bert-score sentence-transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## 평가지표 포함

In [1]:
import os
import json
import re
import warnings
from collections import Counter

import numpy as np
import pandas as pd  # 필요 시 사용
import kss

# datetime: 일관되게 from-import 사용
from datetime import datetime, timezone, timedelta

# NLP / embedding / LLM libs
from langchain.chains import LLMChain
from langchain.schema import StrOutputParser
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

from sentence_transformers import SentenceTransformer, util
from bert_score import score as bert_score

warnings.filterwarnings("ignore")

API_KEY = "your-api-key"

# ========================
# FAISS 벡터DB 로드
# ========================
def load_vectorstore(api_key):
    embedding_model = OpenAIEmbeddings(
        model="text-embedding-3-large",
        openai_api_key=api_key
    )
    vectorstore = FAISS.load_local(
        "vectorDB/faiss_vectorDB",
        embeddings=embedding_model,
        allow_dangerous_deserialization=True
    )
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 3}
    )
    return retriever, vectorstore, embedding_model


# ========================
# ---- 유사도 평가 유틸 ----
# ========================
def _tokens(s):
    s = s.lower()
    s = re.sub(r"[^0-9a-z가-힣\s]", " ", s)
    return s.split()

def prf1_unigram(hyp, ref):
    ht, rt = Counter(_tokens(hyp)), Counter(_tokens(ref))
    inter = sum((ht & rt).values())
    prec = inter / max(1, sum(ht.values()))
    rec = inter / max(1, sum(rt.values()))
    f1 = 0 if (prec + rec) == 0 else 2 * prec * rec / (prec + rec)
    return prec, rec, f1

try:
    from rouge_score import rouge_scorer
    _rouge = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    def rouge_metrics(hyp, ref):
        s = _rouge.score(ref, hyp)
        return {
            "rouge1_f": s["rouge1"].fmeasure,
            "rouge2_f": s["rouge2"].fmeasure,
            "rougeL_f": s["rougeL"].fmeasure
        }
except Exception:
    def rouge_metrics(hyp, ref):
        prec, rec, f1 = prf1_unigram(hyp, ref)
        return {"rouge1_f": f1, "rouge2_f": None, "rougeL_f": None}

def get_topk_refs(text, retriever, k=3):
    docs = retriever.invoke(text)[:k]
    return [d.page_content for d in docs]


[Kss]: From C:\Users\songa\AppData\Local\Programs\Python\Python311\Lib\site-packages\tf_keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.

W1102 23:11:17.564000 2604 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [2]:
# 전역에서 한 번만 로드
_embedding_model = SentenceTransformer("intfloat/multilingual-e5-small")

def score_bot_utterance(bot_utterance, retriever, k=3):
    """
    챗봇 발화(bot_utterance)와 의미적으로 가장 유사한 문장을 찾고
    코사인 유사도 + BERTScore로 평가 (ROUGE 제거)
    """
    # 1) 유사 문서 검색 (FAISS)
    docs = retriever.invoke(bot_utterance)[:k]
    if not docs:
        return None

    # 2) 후보 문장 풀 생성
    candidates = []
    for d in docs:
        sents = _split_sentences(d.page_content)
        candidates.extend(sents)
    if not candidates:
        return None

    # 3) 코사인 유사도 (sentence-transformers)
    q_emb = _embedding_model.encode(bot_utterance, normalize_embeddings=True)
    cand_embs = _embedding_model.encode(candidates, normalize_embeddings=True)
    sims = util.cos_sim(q_emb, cand_embs).squeeze(0).tolist()
    best_idx = max(range(len(sims)), key=lambda i: sims[i])
    best_sent = candidates[best_idx]
    cosine_sim = float(sims[best_idx])

    # 4) BERTScore (ko)
    P, R, F1 = bert_score([bot_utterance], [best_sent], lang="ko", rescale_with_baseline=True)
    bert_p = float(P[0]); bert_r = float(R[0]); bert_f1 = float(F1[0])

    # 5) 반환
    return {
        "ref": best_sent,
        "f1": cosine_sim,
        "cosine": cosine_sim,
        "bertscore": {"bert_p": bert_p, "bert_r": bert_r, "bert_f1": bert_f1},
    }


In [3]:
# ========================
# LLM 프롬프트/체인 준비 (RAG용 retrieved_conversations 제거)
# ========================
def prepare_greeting_llm(api_key):
    llm = ChatOpenAI(
        model="gpt-4o",
        temperature=0.5,
        openai_api_key=api_key
    )

    greeting_system_prompt = """
    너는 아동청소년의 상담 챗봇이야.
    - 아동이 편안하게 느끼도록 따뜻하고 부드러운 반말을 사용해.
    - 대화의 첫 시작이니 긴장하지 않도록 가볍고 친근하게 인사해.
    - 너의 목표는 네가 질문을 진행하기 전 아동에게 친근히 다가가 아동이 질문을 받을 준비가 되었는지 물어보는 거야.
    - 절대! 위험 주제(학대, 자살 등)와 관련된 내용은 언급하지 마.
    - 즐거운 대화라는 말을 하지마.
    - 진중한 대화를 하게될 거야. 아동에겐 편하게 말해달라고 해도 좋아.
    - 너는 일방적으로 질문만 하게 될거고 아동이 하는 질문은 받지 않을 계획이니까 궁금한 점을 물어봐라는 말은 하지마.
    - 질문할 때 한 대화에 같은 의미의 말 여러 번 반복하지마.
    - 1~2문장 정도로 짧게만 말해.
    """

    greeting_human_prompt = """
    --- 현재 대화 맥락 ---
    - 이번 세션 대화 기록:
    {conversation_history}
    - 아동의 최근 답변: {user_response}

    - 상담 준비 질문을 해줘: {ask_ready} (True/False)

    위 모든 맥락을 참고해서 다음 답변을 생성해줘.
    """

    greeting_prompt = ChatPromptTemplate.from_messages([
        ("system", greeting_system_prompt),
        ("human", greeting_human_prompt)
    ])

    return greeting_prompt, llm

def prepare_readiness_judge_llm(api_key):
    llm = ChatOpenAI(
        model="gpt-4o",
        temperature=0,
        openai_api_key=api_key
    )

    judge_system_prompt = """
    너는 아동청소년 상담의 시작 준비 상태를 판단하는 전문가야.
    주어진 대화 내용을 분석하여 아동이 심층적인 상담 주제를 이어갈 준비가 되었는지 판단해줘.

    [판단 기준]
    1. 챗봇이 마지막으로 "준비가 되었는지"에 대해 물어봤고, 아동이 이에 긍정적/적극적으로 응답하며 참여 의지를 보였는가?
    2. 부정적이거나 회피적인 응답이 주를 이루지 않았는가?

    [출력]
    - STOP 또는 KEEP 한 단어만 출력.
    """

    judge_human_prompt = """
    - 최근까지의 초기 대화 내용:
    {initial_conversation_history}

    위 대화를 바탕으로, 아동이 심층적인 상담을 시작할 준비가 되었는지 신중하게 판단해줘.
    STOP 또는 KEEP 한 단어만 출력해.
    """

    judge_prompt = ChatPromptTemplate.from_messages([
        ("system", judge_system_prompt),
        ("human", judge_human_prompt)
    ])

    return judge_prompt, llm

def prepare_first_question_llm(api_key):
    llm = ChatOpenAI(
        model="gpt-4o",
        temperature=0.9,
        openai_api_key=api_key
    )

    first_question_system_prompt = """
    너는 아동청소년의 위험 상태를 파악하기 위한 전문 상담 챗봇이야. 너는 아동과 같은 나이의 또래야.
    - 주어진 카테고리(category)에 맞는 첫 질문을 자연스럽게 시작해.
    - 같은 주제라도 매번 똑같은 질문을 반복하지 마.
    - 아동이 긴장하지 않도록 따뜻하고 부드럽게 반말을 사용해.
    - 너무 깊거나 위협적인 질문은 피하고, 오픈형 질문으로 시작해.
    - 반드시 한 문장으로 작성해.
    - 지난 날 나눴던 대화 기록을 참고해서 질문을 해.
    """

    first_question_human_prompt = """
    - 현재 상담 주제: "{category}"
    위의 주제와 맥락을 참고해서 자연스럽게 첫 질문을 작성해줘.
    """

    first_question_prompt = ChatPromptTemplate.from_messages([
        ("system", first_question_system_prompt),
        ("human", first_question_human_prompt)
    ])

    return first_question_prompt, llm

def prepare_followup_question_llm(api_key):
    """
    RAG용 retrieved_conversations 항목을 프롬프트에서 제거함.
    """
    llm = ChatOpenAI(
        model="gpt-4o",
        temperature=0.3,
        openai_api_key=api_key
    )

    question_system_prompt = """
    너는 아동청소년의 위험 상태를 파악하기 위한 전문 상담 챗봇이야.
    - 아동이 편안하게 답할 수 있도록 부드러운 반말을 사용해.
    - 절대!!! 두 가지 이상의 질문을 한 번에 하지 마. 하나의 질문만 해!!!
    - 만 7세~12살 아동에게 맞는 쉬운 말을 사용하고 부정적인 표현은 피해.
    - 현재 상담 주제와 관련된 말만 해야 해.
    - 아동의 답변이 이해가 되지 않는다면 다른말로 너가 궁금한 점을 포함해서 한번 더 질문해.
    - 공감을 많이 하면서 질문해봐.
    - 아동의 감정을 이끌어낼 수 있도록 노력해봐.
    """

    question_human_prompt = """
    - 현재 상담 주제: "{category}"

    - 이번 세션 대화 기록:
    {conversation_history}

    - 아동의 최근 답변: {user_response}

    위 모든 맥락을 참고해서 아동에게 이어질 질문을 자연스럽게 작성해줘.
    """

    question_prompt = ChatPromptTemplate.from_messages([
        ("system", question_system_prompt),
        ("human", question_human_prompt)
    ])

    return question_prompt, llm

def prepare_judge_llm(api_key):
    llm = ChatOpenAI(
        model="gpt-4o",
        temperature=0,
        openai_api_key=api_key
    )

    judge_system_prompt = """
    너는 아동청소년의 위험 상태를 판단하는 전문가야.
    현재 주제로 상담을 계속할지 또는 전환할지 판단해서 KEEP 또는 SWITCH 한 단어만 출력해.
    """

    judge_human_prompt = """
    - 현재 주제: "{category}"
    - 최근까지의 상담 대화 내용:
    {conversation_history}

    위 대화를 바탕으로, KEEP 또는 SWITCH 중 하나의 단어만 출력해.
    """

    judge_prompt = ChatPromptTemplate.from_messages([
        ("system", judge_system_prompt),
        ("human", judge_human_prompt)
    ])

    return judge_prompt, llm


In [4]:
# ========================
# 전체 초기화 함수
# ========================
def init_chatbot(api_key):
    retriever, vectorstore, embedding_model = load_vectorstore(api_key)
    greeting_prompt, greeting_llm = prepare_greeting_llm(api_key)
    readiness_judge_prompt, readiness_judge_llm = prepare_readiness_judge_llm(api_key)
    first_question_prompt, first_question_llm = prepare_first_question_llm(api_key)
    question_prompt, question_llm = prepare_followup_question_llm(api_key)
    judge_prompt, judge_llm = prepare_judge_llm(api_key)

    greeting_chain = greeting_prompt | greeting_llm | StrOutputParser()
    readiness_judge_chain = readiness_judge_prompt | readiness_judge_llm | StrOutputParser()
    first_question_chain = first_question_prompt | first_question_llm | StrOutputParser()
    question_chain = question_prompt | question_llm | StrOutputParser()
    judge_chain = judge_prompt | judge_llm | StrOutputParser()

    return {
        "retriever": retriever,
        "vectorstore": vectorstore,
        "embedding_model": embedding_model,
        "greeting_chain": greeting_chain,
        "readiness_judge_chain": readiness_judge_chain,
        "first_question_chain": first_question_chain,
        "question_chain": question_chain,
        "judge_chain": judge_chain,
    }

# ========================
# 체인 래퍼 함수 (RAG 관련 인자 제거)
# ========================
def ask_greeting(conversation_history, user_response, ask_ready):
    return chatbot["greeting_chain"].invoke({
        "conversation_history": conversation_history,
        "user_response": user_response,
        "ask_ready": ask_ready
    })

def judge_readiness(initial_conversation_history):
    return chatbot["readiness_judge_chain"].invoke({
        "initial_conversation_history": initial_conversation_history
    }).strip()

def ask_first_question(category):
    return chatbot["first_question_chain"].invoke({"category": category})

# ask_followup_question: 더 이상 retriever에서 꺼낸 텍스트를 LLM에 주입하지 않음.
def ask_followup_question(category, conversation_history, user_response):
    return chatbot["question_chain"].invoke({
        "category": category,
        "conversation_history": conversation_history,
        "user_response": user_response
    })

def should_switch_topic(category, conversation_history):
    return chatbot["judge_chain"].invoke({
        "category": category,
        "conversation_history": conversation_history
    }).strip()



In [5]:
def _split_sentences(text: str):
    try:
        import kss
        return list(kss.split_sentences(text))
    except Exception:
        pattern = r'.+?(?:다\.|요\.|[.!?])(?:\s+|$)'
        sents = re.findall(pattern, text.replace('\r', '\n'), flags=re.DOTALL)
        return [s.strip() for s in sents if s and s.strip()]

def _strip_speaker_prefix(s):
    return re.sub(r"^(상담사|상담자|상담원|아동|아이|child|counselor)\s*[:：]\s*", "", s, flags=re.IGNORECASE).strip()

def _best_sentence_against(hyp, sentences):
    best = None
    best_score = -1.0
    for s in sentences:
        s_clean = _strip_speaker_prefix(s)
        _, _, f1 = prf1_unigram(hyp, s_clean)
        if f1 > best_score:
            best_score = f1
            best = s_clean
    return best, best_score
    

In [22]:
# ========================
# 챗봇 대화 루프 + (의미기반) 유사도 평가 출력/저장
# ========================
def run_conversation(category_list, max_turns=5):
    full_conversation = []
    retriever = chatbot["retriever"]

    if category_list:
        initial_conversation_history = "[시작]\n"

        initial_question = ask_greeting("", "", False)
        initial_question = initial_question.replace("\n", " ").strip()
        print(f"🤖 챗봇: {initial_question}")
        initial_conversation_history += f"상담사: {initial_question}\n"

        # === 유사도 평가 (첫 인사)
        metrics = score_bot_utterance(initial_question, retriever)
        if metrics:
            print("   ↳ 유사문:", metrics['ref'][:100])
            cos = metrics.get("cosine", metrics.get("f1", 0.0))
            print("   ↳ Cosine:", round(cos, 3))
            bs = metrics.get("bertscore")
            if isinstance(bs, dict) and "bert_f1" in bs:
                print(f"   ↳ BERTScore(F1): {bs['bert_f1']:.3f} (P:{bs['bert_p']:.3f}, R:{bs['bert_r']:.3f})")

        user_response_greeting = input("🧒 아동: ")
        initial_conversation_history += f"아동: {user_response_greeting}\n"

        ready_question = ask_greeting(initial_conversation_history.strip(), user_response_greeting, True)
        ready_question = ready_question.replace("\n", " ").strip()
        print(f"🤖 챗봇: {ready_question}")
        initial_conversation_history += f"상담사: {ready_question}\n"

        # === 유사도 평가 (준비 질문)
        metrics = score_bot_utterance(ready_question, retriever)
        if metrics:
            print("   ↳ 유사문:", metrics['ref'][:100])
            cos = metrics.get("cosine", metrics.get("f1", 0.0))
            print("   ↳ Cosine:", round(cos, 3))
            bs = metrics.get("bertscore")
            if isinstance(bs, dict) and "bert_f1" in bs:
                print(f"   ↳ BERTScore(F1): {bs['bert_f1']:.3f} (P:{bs['bert_p']:.3f}, R:{bs['bert_r']:.3f})")

        user_response_ready = input("🧒 아동: ")
        initial_conversation_history += f"아동: {user_response_ready}\n"

        readiness_judge = judge_readiness(initial_conversation_history)
        print("DEBUG: initial_conversation_history:\n", initial_conversation_history)
        print("DEBUG: raw readiness_judge:", repr(readiness_judge))
        
        if readiness_judge == "STOP":
            final_message = "알았어~~ 오늘은 이야기를 나누고 싶지 않구나. 괜찮아! 다음에 이야기하고 싶을 때 언제든지 다시 찾아와줘. 편안한 하루 보내길 바랄게!"
            print(f"\n🤖 챗봇: {final_message}")
            initial_conversation_history += f"상담사: {final_message}\n"
            full_conversation.append(initial_conversation_history)
            return full_conversation

        full_conversation.append(initial_conversation_history)

    # === 주제별 상담
    for category in category_list:
        print(f"\n=== 상담 주제: [{category}] ===")
        full_conversation.append(f"[{category}]\n")
        conversation_history = ""
        turn_count = 0

        current_question = ask_first_question(category)
        current_question = current_question.replace("\n", " ").strip()
        print(f"🤖 챗봇: {current_question}")
        conversation_history += f"상담사: {current_question}\n"

        while turn_count < max_turns:
            user_response = input("🧒 아동: ")
            conversation_history += f"아동: {user_response}\n"

            if user_response.strip() == "종료":
                full_conversation.append(conversation_history)
                break

            judge = should_switch_topic(category, conversation_history)
            if judge == "SWITCH":
                break

            current_question = ask_followup_question(category, conversation_history, user_response)
            current_question = current_question.replace("\n", " ").strip()
            print(f"🤖 챗봇: {current_question}")
            conversation_history += f"상담사: {current_question}\n"

            # === 유사도 평가 (후속 질문)
            metrics = score_bot_utterance(current_question, retriever)
            if metrics:
                print("   ↳ 유사문:", metrics['ref'][:100])
                cos = metrics.get("cosine", metrics.get("f1", 0.0))
                print("   ↳ Cosine:", round(cos, 3))
                bs = metrics.get("bertscore")
                if isinstance(bs, dict) and "bert_f1" in bs:
                    print(f"   ↳ BERTScore(F1): {bs['bert_f1']:.3f} (P:{bs['bert_p']:.3f}, R:{bs['bert_r']:.3f})")

                # ✅ 유사문 + 점수 CSV 저장 (코사인, BERTScore 기준)
                try:
                    with open(save_path, "a", encoding="utf-8") as f:
                        bert_f1 = bs["bert_f1"] if isinstance(bs, dict) else ""
                        safe_ref = metrics["ref"].replace("\n", " ").replace(",", "，")
                        safe_q = current_question.replace("\n", " ").replace(",", "，")
                        f.write(f"{category},{safe_q},{safe_ref},{cos},{bert_f1}\n")
                except Exception as e:
                    print(f"   ↳ (CSV 저장 오류) {e}")

            turn_count += 1

        full_conversation.append(conversation_history)

    print("🤖 챗봇: 오늘 여러 이야기를 나눠줘서 정말 고마워! 편안한 하루 보내길 바랄게!")
    return full_conversation



In [23]:
# === 저장 경로 설정 ===
import os
from datetime import datetime, timezone, timedelta

KST = timezone(timedelta(hours=9))
now = datetime.now(KST).strftime("%Y-%m-%d_%H-%M")

os.makedirs("result_deepchat_no_rag", exist_ok=True)
save_path = f"result_deepchat_no_rag/conversation_deep_{now}.csv"


In [24]:
ROLE_PAT = re.compile(r'(상담사:|아동:)')

def _parse_turns(text: str):
    """한 블록의 대사를 {role, utterance} 리스트로 파싱"""
    if not text:
        return []
    s = text.replace('\r', '\n').strip()

    # '상담사:' / '아동:' 앞에 줄바꿈을 넣어 안전하게 라인 분리
    s = re.sub(r'\s*(상담사:|아동:)', r'\n\1', s)
    lines = [ln.strip() for ln in s.split('\n') if ln.strip()]

    conv = []
    for ln in lines:
        if ln.startswith('상담사:'):
            utt = ln[len('상담사:'):].strip()
            if utt:
                conv.append({"role": "bot", "utterance": utt})
        elif ln.startswith('아동:'):
            utt = ln[len('아동:'):].strip()
            if utt:
                conv.append({"role": "user", "utterance": utt})
    return conv

def convert_to_json(full_conversation):
    json_conversation = []

    for i in range(0, len(full_conversation), 2):
        cat_raw = (full_conversation[i] or "").strip()
        text_raw = (full_conversation[i + 1] if i + 1 < len(full_conversation) else "") or ""
        text_raw = text_raw.strip()

        # 카테고리 문자열에 대사가 섞여 있으면 분리
        m = ROLE_PAT.search(cat_raw)
        if m:
            category = cat_raw[:m.start()]
            leftover = cat_raw[m.start():]  # 첫 대사부터 끝까지
            # leftover를 본문 앞에 붙여서 함께 파싱
            text_raw = (leftover + ("\n" + text_raw if text_raw else "")).strip()
        else:
            category = cat_raw

        # 대괄호/개행 제거 & 트림
        category = category.replace("[", "").replace("]", "").replace("\n", " ").strip()

        conversation = _parse_turns(text_raw)

        json_conversation.append({
            "category": category,
            "conversation": conversation
        })

    return json_conversation

# ========================
# JSON 저장
# ========================
def save_conversation(json_conversation, folder_name="result_deepchat_no_rag"):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    now = datetime.now()
    date_str = now.strftime("%Y-%m-%d_%H-%M")
    filename = f"{folder_name}/conversation_deep_{date_str}.json"

    with open(filename, "w", encoding="utf-8") as f:
        json.dump(json_conversation, f, ensure_ascii=False, indent=2)

    print(f"💾 대화 기록이 {filename} 에 저장되었습니다.")
    return filename

In [25]:
category_list = [
    "분노/짜증", "형제자매",
    "걱정", "정서학대", "신체학대", "성학대",
    "가정폭력", "학교폭력", "자해/자살", "트라우마", "가출"
]
if __name__ == "__main__":
    category_list = category_list

    chatbot = init_chatbot(API_KEY)
    full_conversation = run_conversation(category_list, max_turns=5)
    json_conversation = convert_to_json(full_conversation)
    save_conversation(json_conversation)

🤖 챗봇: 안녕! 만나서 반가워. 오늘 기분은 어때? 준비가 되면 알려줘.


   ↳ 유사문: 저 만날 때 진짜 환하게 인사해 줘요.
   ↳ Cosine: 0.911
   ↳ BERTScore(F1): 0.718 (P:0.705, R:0.731)


🧒 아동:  준비됐어.


🤖 챗봇: 좋아! 그럼 시작해 볼까? 편하게 말해줘도 돼.


   ↳ 유사문: 좋아, 아니면 싫어?
   ↳ Cosine: 0.902
   ↳ BERTScore(F1): 0.743 (P:0.713, R:0.775)


...


=== 상담 주제: [트라우마] ===
🤖 챗봇: 트라우마라는 게 참 복잡한 감정인데, 혹시 최근에 특별히 마음에 남는 일이 있었어?


🧒 아동:  아니



=== 상담 주제: [가출] ===
🤖 챗봇: 집이 아닌 다른 곳에서 잠을 자본 적 있어?


🧒 아동:  아니


🤖 챗봇: 오늘 여러 이야기를 나눠줘서 정말 고마워! 편안한 하루 보내길 바랄게!
💾 대화 기록이 result_deepchat_no_rag/conversation_deep_2025-11-03_01-12.json 에 저장되었습니다.


#### 